In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [12]:
%env csv_path=/home/jupyter
%env output_dir=/home/jupyter/data
%env data_type=para
%env languages=Bafia,Bulu
! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages

env: csv_path=/home/jupyter
env: output_dir=/home/jupyter/data
env: data_type=para
env: languages=Bafia,Bulu
Bafia - Bulu
======= Read 7950 totals samples
======= Delete 4 samples
======= Save 7946 samples
Bulu
======= Read 31297 totals samples
======= Delete 2402 samples
======= Save 28895 samples


In [20]:
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=/home/jupyter/data/Bafia-Bulu        
%env MONO_PATH=""  
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data_monoxlm.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Bafia-Bulu:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=/home/jupyter/data/Bafia-Bulu
env: MONO_PATH=""
env: SAME_VOCAB=True
env: nCodes=10000
env: shuf_n_samples=1000000
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
env: n_samples=-1
env: sub_task=Bafia-Bulu:-1


In [16]:
%env add_para_data_to_mono_data=False
! ../build_meta_data_monoxlm.sh $sub_task $n_samples $add_para_data_to_mono_data

env: add_para_data_to_mono_data=False
dir /home/jupyter/models/africa/cluster1/data/Bafi/processed/fine_tune already exists
params ok !
*** Cleaning and tokenizing Bafia-Bulu data ... ***
Tokenizer Version 1.1
Language: Bafia
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia-Bulu.Bafia data to /home/jupyter/data/Bafia-Bulu/? ***
Tokenizer Version 1.1
Language: Bulu
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia-Bulu.Bulu data to /home/jupyter/data/Bafia-Bulu/? ***


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error



***build the training set for BPE tokenization (10000 codes)***


***shuf ... Generating 1000000 random permutations of training data and store result in /home/jupyter/models/africa/cluster1/data/Bafi/processed/Bafia-Bulu/bpe.train***


***Lea

In [2]:
%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [3]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [22]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/Bafia-Bulu.Bafia.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6360


In [10]:
%env epoch_size=6360

%env lgs=Bafia-Bulu

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6360
env: lgs=Bafia-Bulu
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [17]:
%env mlm_steps=Bafia,Bulu,Bafia-Bulu
%env batch_size=32
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster1
! python train.py --exp_name mlm_tlm_BafiaBulu --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Bafia,Bulu,Bafia-Bulu
env: batch_size=32
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster1
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/24/20 08:24:53 - 0:00:00 - ============ Initialized logger ============
INFO - 05/24/20 08:24:53 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_size: 1
                           

INFO - 05/24/20 08:24:53 - 0:00:00 - ========================== debug : 4

INFO - 05/24/20 08:24:53 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/valid.Bulu.pth ...
INFO - 05/24/20 08:24:53 - 0:00:00 - 23036 words (8683 unique) in 794 sentences. 61 unknown words (52 unique) covering 0.26% of the data.
INFO - 05/24/20 08:24:53 - 0:00:00 - ========================== debug : 5

INFO - 05/24/20 08:24:53 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/test.Bulu.pth ...
INFO - 05/24/20 08:24:53 - 0:00:00 - 23180 words (8683 unique) in 794 sentences. 59 unknown words (54 unique) covering 0.25% of the data.
INFO - 05/24/20 08:24:53 - 0:00:00 - ========================== debug : 6


INFO - 05/24/20 08:24:53 - 0:00:00 - ============ Parallel data (Bafia-Bulu)
INFO - 05/24/20 08:24:53 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/train.Bafia-Bulu.Bafia.pth ...

INFO - 05/24/20 08:24:57 - 0:00:04 - Found 0 memories.
INFO - 05/24/20 08:24:57 - 0:00:04 - Found 6 FFN.
INFO - 05/24/20 08:24:57 - 0:00:04 - Found 102 parameters in model.
INFO - 05/24/20 08:24:57 - 0:00:04 - Optimizers: model
WARNING - 05/24/20 08:24:57 - 0:00:04 - Reloading checkpoint from /home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/checkpoint.pth ...
WARNING - 05/24/20 08:24:58 - 0:00:05 - Not reloading checkpoint optimizer model.
WARNING - 05/24/20 08:24:58 - 0:00:05 - No 'num_updates' for optimizer model.
WARNING - 05/24/20 08:24:58 - 0:00:05 - Checkpoint reloaded. Resuming at epoch 58 / iteration 3886 ...
INFO - 05/24/20 08:24:59 - 0:00:05 - ============ Starting epoch 58 ... ============
INFO - 05/24/20 08:24:59 - 0:00:05 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/24/20 08:25:01 - 0:00:08 - Creating new training data iterator (pred,Bafia,Bulu) ...
INFO - 05/24/20 08:25:04 - 0:00:11 - Creating new training data iterator (pred,Bulu) ...
INFO 

INFO - 05/24/20 08:30:55 - 0:06:02 -    4005 -   31.78 sent/s - 22231.63 words/s - MLM-Bafia:  0.9800 || MLM-Bulu:  0.8454 || MLM-Bafia-Bulu:  1.1746 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:31:10 - 0:06:17 -    4010 -   31.96 sent/s - 23303.69 words/s - MLM-Bafia:  0.9260 || MLM-Bulu:  0.8652 || MLM-Bafia-Bulu:  1.1533 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:31:25 - 0:06:32 -    4015 -   32.53 sent/s - 24604.35 words/s - MLM-Bafia:  0.9501 || MLM-Bulu:  0.8213 || MLM-Bafia-Bulu:  1.2225 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:31:39 - 0:06:46 -    4020 -   33.50 sent/s - 26291.26 words/s - MLM-Bafia:  1.0139 || MLM-Bulu:  0.8317 || MLM-Bafia-Bulu:  1.1227 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:31:39 - 0:06:46 - ============ End of epoch 59 ============
INFO - 05/24/20 08:31:42 - 0:06:49 - epoch -> 59.000000
INFO - 05/24/20 08:31:42 - 0:06:49 - valid_Bafia_mlm_ppl -> 7.950344
INFO - 05/24/20 08:31:42 - 0:06:49 - valid_Bafia_mlm_acc -> 59.585492
INFO - 05/24/20 08:31:

INFO - 05/24/20 08:35:17 - 0:10:24 - ============ garbage collector collecting 0 ...
INFO - 05/24/20 08:35:17 - 0:10:24 - ============ Starting epoch 61 ... ============
INFO - 05/24/20 08:35:23 - 0:10:30 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/24/20 08:35:26 - 0:10:33 -    4090 -   20.01 sent/s - 23896.64 words/s - MLM-Bafia:  0.9345 || MLM-Bulu:  0.8310 || MLM-Bafia-Bulu:  1.1327 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:35:41 - 0:10:48 -    4095 -   32.15 sent/s - 39267.33 words/s - MLM-Bafia:  0.9597 || MLM-Bulu:  0.8120 || MLM-Bafia-Bulu:  1.1942 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:35:44 - 0:10:51 - Creating new training data iterator (pred,Bulu) ...
INFO - 05/24/20 08:35:56 - 0:11:03 -    4100 -   31.85 sent/s - 39807.89 words/s - MLM-Bafia:  0.9479 || MLM-Bulu:  0.8487 || MLM-Bafia-Bulu:  1.1523 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:36:11 - 0:11:18 -    4105 -   30.89 sent/s - 39711.87 words/s - MLM-Bafia:  0.9163 || MLM-Bulu:  0.7972 

INFO - 05/24/20 08:42:03 - 0:17:09 -    4220 -   32.55 sent/s - 63506.27 words/s - MLM-Bafia:  0.9194 || MLM-Bulu:  0.7324 || MLM-Bafia-Bulu:  1.0627 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:42:06 - 0:17:12 - ============ End of epoch 62 ============
INFO - 05/24/20 08:42:08 - 0:17:15 - epoch -> 62.000000
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_Bafia_mlm_ppl -> 8.297229
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_Bafia_mlm_acc -> 60.362694
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_Bulu_mlm_ppl -> 17.584191
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_Bulu_mlm_acc -> 50.777202
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_Bafia_Bulu_mlm_ppl -> 11.574270
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_Bafia_Bulu_mlm_acc -> 54.257028
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_mlm_ppl -> 12.940710
INFO - 05/24/20 08:42:08 - 0:17:15 - valid_mlm_acc -> 55.569948
INFO - 05/24/20 08:42:08 - 0:17:15 - test_Bafia_mlm_ppl -> 8.412698
INFO - 05/24/20 08:42:08 - 0:17:15 - test_Bafia_mlm_acc -> 51.2953

INFO - 05/24/20 08:46:19 - 0:21:26 -    4300 -   32.45 sent/s - 78413.63 words/s - MLM-Bafia:  0.9085 || MLM-Bulu:  0.7800 || MLM-Bafia-Bulu:  1.0368 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:46:19 - 0:21:26 - Creating new training data iterator (pred,Bulu) ...
INFO - 05/24/20 08:46:32 - 0:21:39 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/24/20 08:46:34 - 0:21:41 -    4305 -   32.17 sent/s - 78832.33 words/s - MLM-Bafia:  0.8788 || MLM-Bulu:  0.7397 || MLM-Bafia-Bulu:  1.0095 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:46:49 - 0:21:55 -    4310 -   32.61 sent/s - 80691.66 words/s - MLM-Bafia:  0.8770 || MLM-Bulu:  0.7631 || MLM-Bafia-Bulu:  1.0671 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:47:03 - 0:22:10 -    4315 -   33.45 sent/s - 83738.01 words/s - MLM-Bafia:  0.8860 || MLM-Bulu:  0.7998 || MLM-Bafia-Bulu:  1.0526 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:47:18 - 0:22:25 -    4320 -   31.69 sent/s - 80449.46 words/s - MLM-Bafia:  0.9311 || MLM-Bulu:  0

INFO - 05/24/20 08:52:41 - 0:27:48 - ============ garbage collector collecting 0 ...
INFO - 05/24/20 08:52:41 - 0:27:48 - ============ Starting epoch 66 ... ============
INFO - 05/24/20 08:52:50 - 0:27:57 -    4425 -   20.11 sent/s - 63289.24 words/s - MLM-Bafia:  0.8847 || MLM-Bulu:  0.7215 || MLM-Bafia-Bulu:  1.0131 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:53:05 - 0:28:12 -    4430 -   32.13 sent/s - 102074.90 words/s - MLM-Bafia:  0.8567 || MLM-Bulu:  0.7294 || MLM-Bafia-Bulu:  1.0841 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:53:20 - 0:28:27 -    4435 -   31.15 sent/s - 100096.95 words/s - MLM-Bafia:  0.8773 || MLM-Bulu:  0.7082 || MLM-Bafia-Bulu:  1.1502 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:53:20 - 0:28:27 - Creating new training data iterator (pred,Bulu) ...
INFO - 05/24/20 08:53:35 - 0:28:42 -    4440 -   32.13 sent/s - 103952.88 words/s - MLM-Bafia:  0.8700 || MLM-Bulu:  0.6795 || MLM-Bafia-Bulu:  1.0569 -  - model LR: 1.0000e-04
INFO - 05/24/20 08:53:50 - 0:28:57

INFO - 05/24/20 08:59:33 - 0:34:40 - ============ End of epoch 67 ============
INFO - 05/24/20 08:59:35 - 0:34:42 - epoch -> 67.000000
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_Bafia_mlm_ppl -> 7.829817
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_Bafia_mlm_acc -> 62.176166
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_Bulu_mlm_ppl -> 18.810103
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_Bulu_mlm_acc -> 46.373057
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_Bafia_Bulu_mlm_ppl -> 11.779372
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_Bafia_Bulu_mlm_acc -> 54.819277
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_mlm_ppl -> 13.319960
INFO - 05/24/20 08:59:35 - 0:34:42 - valid_mlm_acc -> 54.274611
INFO - 05/24/20 08:59:35 - 0:34:42 - test_Bafia_mlm_ppl -> 9.104142
INFO - 05/24/20 08:59:35 - 0:34:42 - test_Bafia_mlm_acc -> 53.108808
INFO - 05/24/20 08:59:35 - 0:34:42 - test_Bulu_mlm_ppl -> 46.008533
INFO - 05/24/20 08:59:35 - 0:34:42 - test_Bulu_mlm_acc -> 37.564767
INFO - 05/24/20 08:59:35 - 0:34:42 - 

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [18]:
%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
%env exp_id=maml
%env epoch_size=6360
%env lgs=Bafia-Bulu

env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
env: exp_id=maml
env: epoch_size=6360
env: lgs=Bafia-Bulu


In [19]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [22]:
%env stopping_criterion=valid_Bafia-Bulu_mt_bleu,10
%env validation_metrics=valid_Bafia-Bulu_mt_bleu
%env reload_model=/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth

%env ae_steps=Bafia,Bulu
%env bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu

env: stopping_criterion=valid_Bafia-Bulu_mt_bleu,10
env: validation_metrics=valid_Bafia-Bulu_mt_bleu
env: reload_model=/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
env: ae_steps=Bafia,Bulu
env: bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu


In [4]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
#! python train.py --exp_name UnSupMT_BafiaBulu --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100


In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [23]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100
%env batch_size=32
%env dump_path=/home/jupyter/models/africa/cluster1
# supervised MT
%env mt_steps=Bafia-Bulu,Bulu-Bafia          
! python train.py --exp_name SupMT_BafiaBulu --exp_id $exp_id  --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100
env: batch_size=32
env: dump_path=/home/jupyter/models/africa/cluster1
env: mt_steps=Bafia-Bulu,Bulu-Bafia
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/24/20 09:14:06 - 0:00:00 - ============ Initialized logger ============
INFO - 05/24/20 09:14:06 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Bafia', 'Bulu']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_s

INFO - 05/24/20 09:14:06 - 0:00:00 - ========================== debug : 7
INFO - 05/24/20 09:14:06 - 0:00:00 - ========================== debug : 8
INFO - 05/24/20 09:14:06 - 0:00:00 - Removed 0 empty sentences.
INFO - 05/24/20 09:14:06 - 0:00:00 - Removed 0 too long sentences.

INFO - 05/24/20 09:14:06 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/valid.Bulu.pth ...
INFO - 05/24/20 09:14:06 - 0:00:00 - 23036 words (8683 unique) in 794 sentences. 61 unknown words (52 unique) covering 0.26% of the data.
INFO - 05/24/20 09:14:06 - 0:00:00 - ========================== debug : 9
INFO - 05/24/20 09:14:06 - 0:00:00 - ========================== debug : 10
INFO - 05/24/20 09:14:06 - 0:00:00 - Removed 0 too long sentences.

INFO - 05/24/20 09:14:06 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/test.Bulu.pth ...
INFO - 05/24/20 09:14:06 - 0:00:00 - 23180 words (8683 unique) in 794 sentences. 59 unknown 

INFO - 05/24/20 09:14:11 - 0:00:06 - Number of parameters (encoder): 85005803
INFO - 05/24/20 09:14:11 - 0:00:06 - Number of parameters (decoder): 110208491
INFO - 05/24/20 09:14:12 - 0:00:06 - Found 0 memories.
INFO - 05/24/20 09:14:12 - 0:00:06 - Found 12 FFN.
INFO - 05/24/20 09:14:12 - 0:00:06 - Found 264 parameters in model.
INFO - 05/24/20 09:14:12 - 0:00:06 - Optimizers: model
INFO - 05/24/20 09:14:12 - 0:00:06 - ============ Starting epoch 0 ... ============
INFO - 05/24/20 09:14:12 - 0:00:06 - Creating new training data iterator (ae,Bulu) ...
INFO - 05/24/20 09:14:13 - 0:00:08 - Creating new training data iterator (ae,Bafia) ...
INFO - 05/24/20 09:14:14 - 0:00:08 - Creating new training data iterator (mt,Bulu,Bafia) ...
INFO - 05/24/20 09:14:15 - 0:00:09 - Creating new training data iterator (mt,Bafia,Bulu) ...
INFO - 05/24/20 09:14:15 - 0:00:09 - Creating new training data iterator (bt,Bafia) ...
INFO - 05/24/20 09:14:16 - 0:00:10 - Creating new training data iterator (bt,Bulu

INFO - 05/24/20 09:23:17 - 0:09:12 - BLEU /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/hyp1.Bafia-Bulu.valid.txt /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/ref.Bafia-Bulu.valid.txt : 2.680000
INFO - 05/24/20 09:23:41 - 0:09:36 - BLEU /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/hyp1.Bulu-Bafia.test.txt /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/ref.Bulu-Bafia.test.txt : 2.410000
INFO - 05/24/20 09:24:10 - 0:10:04 - BLEU /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/hyp1.Bafia-Bulu.test.txt /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/ref.Bafia-Bulu.test.txt : 2.690000
INFO - 05/24/20 09:24:10 - 0:10:04 - epoch -> 1.000000
INFO - 05/24/20 09:24:10 - 0:10:04 - valid_Bulu-Bafia_mt_ppl -> 36.276641
INFO - 05/24/20 09:24:10 - 0:10:04 - valid_Bulu-Bafia_mt_acc -> 29.784866
INFO - 05/24/20 09:24:10 - 0:10:04 - valid_Bulu-Bafia_mt_bleu -> 2.340000
I

INFO - 05/24/20 09:30:04 - 0:15:58 - ============ garbage collector collecting 0 ...
INFO - 05/24/20 09:30:04 - 0:15:58 - ============ Starting epoch 3 ... ============
INFO - 05/24/20 09:30:23 - 0:16:17 -     105 -    7.90 sent/s -  6686.47 words/s - AE-Bafia:  3.0409 || AE-Bulu:  3.4017 || MT-Bafia-Bulu:  3.4361 || MT-Bulu-Bafia:  3.1695 || BT-Bafia-Bulu-Bafia:  3.1979 || BT-Bulu-Bafia-Bulu:  3.6626 -  - model LR: 1.5834e-05
INFO - 05/24/20 09:30:54 - 0:16:48 -     110 -   42.94 sent/s - 33797.26 words/s - AE-Bafia:  2.9577 || AE-Bulu:  3.3005 || MT-Bafia-Bulu:  3.5406 || MT-Bulu-Bafia:  3.0531 || BT-Bafia-Bulu-Bafia:  3.1596 || BT-Bulu-Bafia-Bulu:  3.5379 -  - model LR: 1.6583e-05
INFO - 05/24/20 09:31:25 - 0:17:19 -     115 -   46.05 sent/s - 36487.15 words/s - AE-Bafia:  2.9632 || AE-Bulu:  3.2437 || MT-Bafia-Bulu:  3.3552 || MT-Bulu-Bafia:  3.0679 || BT-Bafia-Bulu-Bafia:  3.2119 || BT-Bulu-Bafia-Bulu:  3.4871 -  - model LR: 1.7333e-05
INFO - 05/24/20 09:31:55 - 0:17:49 -     120 

...

INFO - 05/24/20 12:07:54 - 2:53:48 - BLEU /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/hyp23.Bafia-Bulu.test.txt /home/jupyter/models/africa/cluster1/SupMT_BafiaBulu/maml/hypotheses/ref.Bafia-Bulu.test.txt : 9.290000
INFO - 05/24/20 12:07:54 - 2:53:48 - epoch -> 23.000000
INFO - 05/24/20 12:07:54 - 2:53:48 - valid_Bulu-Bafia_mt_ppl -> 18.810276
INFO - 05/24/20 12:07:54 - 2:53:48 - valid_Bulu-Bafia_mt_acc -> 44.099521
INFO - 05/24/20 12:07:54 - 2:53:48 - valid_Bulu-Bafia_mt_bleu -> 11.760000
INFO - 05/24/20 12:07:54 - 2:53:48 - valid_Bafia-Bulu_mt_ppl -> 51.793938
INFO - 05/24/20 12:07:54 - 2:53:48 - valid_Bafia-Bulu_mt_acc -> 35.013330
INFO - 05/24/20 12:07:54 - 2:53:48 - valid_Bafia-Bulu_mt_bleu -> 9.250000
INFO - 05/24/20 12:07:54 - 2:53:48 - test_Bulu-Bafia_mt_ppl -> 19.416068
INFO - 05/24/20 12:07:54 - 2:53:48 - test_Bulu-Bafia_mt_acc -> 43.877350
INFO - 05/24/20 12:07:54 - 2:53:48 - test_Bulu-Bafia_mt_bleu -> 11.450000
INFO - 05/24/20 12:07:54 - 2:53:48 - t

INFO - 05/24/20 12:22:13 - 3:08:08 -     870 -   20.07 sent/s - 142499.21 words/s - AE-Bafia:  0.4724 || AE-Bulu:  0.3998 || MT-Bafia-Bulu:  0.8716 || MT-Bulu-Bafia:  1.2592 || BT-Bafia-Bulu-Bafia:  1.3223 || BT-Bulu-Bafia-Bulu:  1.0661 -  - model LR: 8.7538e-05
INFO - 05/24/20 12:22:48 - 3:08:43 - Creating new training data iterator (ae,Bulu) ...
INFO - 05/24/20 12:22:56 - 3:08:50 - Creating new training data iterator (bt,Bulu) ...
INFO - 05/24/20 12:23:11 - 3:09:05 -     875 -   24.41 sent/s - 146841.23 words/s - AE-Bafia:  0.4618 || AE-Bulu:  0.3555 || MT-Bafia-Bulu:  0.9073 || MT-Bulu-Bafia:  1.2166 || BT-Bafia-Bulu-Bafia:  1.1992 || BT-Bulu-Bafia-Bulu:  0.9803 -  - model LR: 8.7287e-05
INFO - 05/24/20 12:24:09 - 3:10:03 -     880 -   22.73 sent/s - 148341.09 words/s - AE-Bafia:  0.4783 || AE-Bulu:  0.3461 || MT-Bafia-Bulu:  0.8345 || MT-Bulu-Bafia:  1.2442 || BT-Bafia-Bulu-Bafia:  1.1946 || BT-Bulu-Bafia-Bulu:  0.8109 -  - model LR: 8.7039e-05
INFO - 05/24/20 12:24:55 - 3:10:49 - 

INFO - 05/24/20 12:38:35 - 3:24:29 - ============ garbage collector collecting 0 ...
INFO - 05/24/20 12:38:35 - 3:24:29 - ============ Starting epoch 27 ... ============
INFO - 05/24/20 12:39:00 - 3:24:54 -     920 -    4.72 sent/s - 33616.35 words/s - AE-Bafia:  0.4711 || AE-Bulu:  0.3821 || MT-Bafia-Bulu:  0.8130 || MT-Bulu-Bafia:  1.0624 || BT-Bafia-Bulu-Bafia:  1.1626 || BT-Bulu-Bafia-Bulu:  0.8335 -  - model LR: 8.5126e-05
INFO - 05/24/20 12:39:56 - 3:25:50 -     925 -   28.47 sent/s - 159823.58 words/s - AE-Bafia:  0.4720 || AE-Bulu:  0.3204 || MT-Bafia-Bulu:  0.7875 || MT-Bulu-Bafia:  1.1575 || BT-Bafia-Bulu-Bafia:  1.1482 || BT-Bulu-Bafia-Bulu:  0.8822 -  - model LR: 8.4895e-05
INFO - 05/24/20 12:40:54 - 3:26:48 -     930 -   21.19 sent/s - 154399.74 words/s - AE-Bafia:  0.4189 || AE-Bulu:  0.3187 || MT-Bafia-Bulu:  0.9064 || MT-Bulu-Bafia:  1.0863 || BT-Bafia-Bulu-Bafia:  1.2118 || BT-Bulu-Bafia-Bulu:  0.7837 -  - model LR: 8.4667e-05
INFO - 05/24/20 12:41:51 - 3:27:45 -     9

INFO - 05/24/20 12:59:15 - 3:45:09 - ============ garbage collector collecting 0 ...
INFO - 05/24/20 12:59:15 - 3:45:09 - ============ Starting epoch 29 ... ============
INFO - 05/24/20 13:00:00 - 3:45:54 -     990 -    5.17 sent/s - 34507.78 words/s - AE-Bafia:  0.4151 || AE-Bulu:  0.2985 || MT-Bafia-Bulu:  0.4524 || MT-Bulu-Bafia:  0.7900 || BT-Bafia-Bulu-Bafia:  1.1089 || BT-Bulu-Bafia-Bulu:  0.6363 -  - model LR: 8.2061e-05
INFO - 05/24/20 13:00:47 - 3:46:41 - Creating new training data iterator (ae,Bafia) ...
INFO - 05/24/20 13:00:56 - 3:46:51 - Creating new training data iterator (bt,Bafia) ...
INFO - 05/24/20 13:00:59 - 3:46:54 -     995 -   21.26 sent/s - 163100.44 words/s - AE-Bafia:  0.4009 || AE-Bulu:  0.2452 || MT-Bafia-Bulu:  0.4696 || MT-Bulu-Bafia:  0.7438 || BT-Bafia-Bulu-Bafia:  1.0857 || BT-Bulu-Bafia-Bulu:  0.6230 -  - model LR: 8.1855e-05
INFO - 05/24/20 13:01:55 - 3:47:49 -    1000 -   27.00 sent/s - 174383.06 words/s - AE-Bafia:  0.3390 || AE-Bulu:  0.2483 || MT-B

INFO - 05/24/20 13:19:19 - 4:05:13 - ============ garbage collector collecting 0 ...
INFO - 05/24/20 13:19:19 - 4:05:13 - ============ Starting epoch 31 ... ============
INFO - 05/24/20 13:19:29 - 4:05:23 -    1055 -    5.39 sent/s - 36634.33 words/s - AE-Bafia:  0.3437 || AE-Bulu:  0.2341 || MT-Bafia-Bulu:  0.4538 || MT-Bulu-Bafia:  0.7001 || BT-Bafia-Bulu-Bafia:  0.8517 || BT-Bulu-Bafia-Bulu:  0.6018 -  - model LR: 7.9493e-05
INFO - 05/24/20 13:20:28 - 4:06:22 -    1060 -   21.40 sent/s - 175908.97 words/s - AE-Bafia:  0.3013 || AE-Bulu:  0.2334 || MT-Bafia-Bulu:  0.4312 || MT-Bulu-Bafia:  0.7020 || BT-Bafia-Bulu-Bafia:  0.8406 || BT-Bulu-Bafia-Bulu:  0.6081 -  - model LR: 7.9305e-05
INFO - 05/24/20 13:21:25 - 4:07:19 -    1065 -   22.65 sent/s - 180701.30 words/s - AE-Bafia:  0.3155 || AE-Bulu:  0.2418 || MT-Bafia-Bulu:  0.4552 || MT-Bulu-Bafia:  0.7338 || BT-Bafia-Bulu-Bafia:  0.8504 || BT-Bulu-Bafia-Bulu:  0.5692 -  - model LR: 7.9119e-05
INFO - 05/24/20 13:21:49 - 4:07:44 - Creat

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1